In [ ]:
using Pkg; Pkg.add("FortranFiles"); Pkg.add("LinearAlgebra")
using FortranFiles
using LinearAlgebra

In [ ]:
# Create solution, sing structs
struct solution_type
    msol :: Int32
    u :: Array{ComplexF64,3}
end

struct resist_type
    e :: Float64
    f :: Float64
    h :: Float64
    m :: Float64
    g :: Float64
    k :: Float64
    eta :: Float64
    rho :: Float64
    taua :: Float64
    taur :: Float64
    di :: Float64
    dr :: Float64
    sfac :: Float64
    deltac :: Float64
    eigenvalue :: ComplexF64
end

struct sing_type
    msol_l :: Int32
    msol_r :: Int32
    jfix :: Int32
    jpert :: Int32
    psifac :: Float64
    q :: Float64
    q1 :: Float64
    ca_l :: Array{ComplexF64,3}
    ca_r :: Array{ComplexF64,3}
    restype :: resist_type
end

mutable struct fixfac_type
    msol :: Int32
    index :: Vector{Int32}
    fixfac :: Array{ComplexF64,2}
    transform :: Array{ComplexF64,2}
    gauss :: Array{ComplexF64,2}
    function fixfac_type(msol::Int32, index::Vector{Int32},fixfac::Array{ComplexF64,2})
        transform = Matrix{ComplexF64}(I,msol,msol)
        gauss = Matrix{ComplexF64}(I,msol,msol)
        new(msol,index,fixfac,transform,gauss)
    end
end

function idcon_transform(fixtype, sing_flag)
    
    msol = length(fixtype[2].index)
    mfix = length(fixtype)-1
    identity = Matrix{ComplexF64}(I,msol,msol)

    # Construct gaussian reduction matrix
    for ifix in 2:1:mfix+1
        fixtype[ifix].gauss = copy(identity)
        mask = zeros(Bool, msol)
        mask .= true
        for isol in 1:msol
            ksol = fixtype[ifix].index[isol]
            mask[ksol] = false
            temp = copy(identity)
            for jsol in 1:msol
                if mask[jsol]
                    temp[ksol,jsol] = fixtype[ifix].fixfac[ksol,jsol]
                end
            end
            # Matrix multiplication gauss = gauss * temp
            fixtype[ifix].gauss = fixtype[ifix].gauss * temp
        end
        if sing_flag[ifix-1]
            fixtype[ifix].gauss[:,fixtype[ifix].index[1]] .= 0.0
        end
    end
    
    # Concatenate gaussian reduction matrix
    fixtype[mfix+1].transform = copy(identity)
    for ifix in mfix:-1:1
        fixtype[ifix].transform = fixtype[ifix+1].gauss * fixtype[ifix+1].transform
    end
    return fixtype
end
    

In [ ]:
fstream = FortranFile("euler.bin", "r")

# --- Read header ---
mlow, mhigh, nn, mpsi, mtheta, ro, zo = read(fstream, Int32, Int32, Int32, Int32, Int32, Float64, Float64)
mband, mthsurf0, mthvac, psio, psilow, psilim, qlim, singfac_min = read(fstream, Int32, Int32, Int32, Float64, Float64, Float64, Float64, Float64)
power_b, power_r, power_bp = read(fstream, Int32, Int32, Int32)
kin_flag, con_flag = read(fstream, Int32, Int32)

amean, rmean, aratio, kappa, delta1, delta2, 
    li1, li2, li3, betap1, betap2, betap3, betat, betan, bt0, 
    q0, qmin, qmax, qa, crnt, q95, shotnum, shottime = read(fstream, 
        Float64, Float64, Float64, Float64, Float64, Float64,
        Float64, Float64, Float64, Float64, Float64, Float64,
        Float64, Float64, Float64, Float64, Float64, Float64,
        Float64, Float64, Float64, Float64, Float64)

# --- Read equilibrium on flux coordinates ---
# sq%xs is size mpsi, sq%fs is size mpsi x 4, sq%fs1 is size mpsi x 4, sq%xpower is size 2 x 4
sq_xs, sq_fs, sq_fs1, sq_xpower = read(fstream, (Float64, mpsi), (Float64, (mpsi,4)), (Float64, (mpsi,4)), (Float64, (2,4)))
# rzphi%xs is size mpsi, rzphi%ys is size mtheta, rzphi%fs is size mpsi x mtheta, etc.
rzphi_xs, rzphi_ys, 
    rzphi_fs, rzphi_fsx, rzphi_fsy, rzphi_fsxy, 
    rzphi_x0, rzphi_y0, rzphi_xpower, rzphi_ypower = read(fstream, 
        (Float64, mpsi), (Float64, mtheta), 
        (Float64, (mpsi,mtheta,4)), (Float64, (mpsi,mtheta,4)), (Float64, (mpsi,mtheta,4)), (Float64, (mpsi,mtheta,4)),
        (Float64, 2), (Float64, 2), (Float64, (2,4)), (Float64, (2,4)))

# misc derived quantities
chi1=2*pi*psio
mpert=mhigh-mlow+1

# --- Count solutions in file (first pass) ---
mstep = -1
mfix = 0
msing = 0

while !eof(fstream)
    data_type = read(fstream, Int32)
    if data_type == 1
        mstep += 1
        read(fstream)  # skip
        read(fstream)
        read(fstream)
    elseif data_type == 2
        mfix += 1
        read(fstream)
        read(fstream)
    elseif data_type == 3
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
    elseif data_type == 4
        msing += 1
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
    elseif data_type == 5
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
    else
        error("Cannot recognize data_type = $data_type")
    end
end

close(fstream)

psifac = zeros(Float64, mstep+1)
rhofac = zeros(Float64, mstep+1)
qfac = zeros(Float64, mstep+1)

soltype = Vector{solution_type}(undef, mstep+1)
singtype = Vector{sing_type}(undef, msing)
fixstep = zeros(Float64, mfix+2)
fixtype = Vector{fixfac_type}(undef, mfix+1)
sing_flag = zeros(Bool, mfix)

fixtype[1] = fixfac_type(Int32(0), zeros(Int32, 0), zeros(ComplexF64, 0, 0))

et = zeros(ComplexF64, mpert)
ep = zeros(ComplexF64, mpert)
ee = zeros(ComplexF64, mpert)

wt = zeros(ComplexF64, (mpert,mpert))
wt0 = zeros(ComplexF64, (mpert,mpert))

wft = zeros(ComplexF64, (mpert,mpert))
eft = zeros(ComplexF64, mpert)
efp = zeros(ComplexF64, mpert)
wtraw = zeros(ComplexF64, (mpert,mpert))

eft = -1
wft = 0
fixstep[1] = 0
fixstep[mfix+2] = mstep

fstream = FortranFile("euler.bin", "r")
# --- Read header again ---
mlow, mhigh, nn, mpsi, mtheta, ro, zo = read(fstream, Int32, Int32, Int32, Int32, Int32, Float64, Float64)
mband, mthsurf0, mthvac, psio, psilow, psilim, qlim, singfac_min = read(fstream, Int32, Int32, Int32, Float64, Float64, Float64, Float64, Float64)
power_b, power_r, power_bp = read(fstream, Int32, Int32, Int32)
kin_flag, con_flag = read(fstream, Int32, Int32)
istep = -1
ifix = 0
ising = 0

println("mfix = ", mfix)

while !eof(fstream)
    data_type = read(fstream, Int32)
    if data_type == 1
        istep += 1
        psifac[istep+1], qfac[istep+1], msoli = read(fstream, Float64, Float64, Int32)
        u = Array{ComplexF64}(undef,(mpert,msoli,4))
        u[:,:,1:2] = read(fstream,  (ComplexF64,(mpert,msoli,2)))
        u[:,:,3:4] = read(fstream,  (ComplexF64,(mpert,msoli,2)))
        soltype[istep+1] = solution_type(msoli,u)
    elseif data_type == 2
        ifix += 1
        println("Writing to fixtype[", ifix+1, "]")
        fixstep[ifix+1] = istep
        sing_flag[ifix], msoli = read(fstream, Int32, Int32)
        fixfac, index = read(fstream, (ComplexF64,(msoli,msoli)),(Int32,msoli))
        fixtype[ifix+1] = fixfac_type(msoli,index,fixfac)
    elseif data_type == 3
        ep = read(fstream,(ComplexF64,mpert))
        et = read(fstream,(ComplexF64,mpert))
        wt = read(fstream,(ComplexF64,(mpert,mpert)))
        wt0 = read(fstream,(ComplexF64,(mpert,mpert)))
        wv_farwall_flag = read(fstream, Int32)
    elseif data_type == 4
        ising += 1
        jfixi = ifix
        jperti = round(Int32, nn*qfac[istep+1] + 0.5)
        psifaci, qi, q1i = read(fstream, Float64, Float64, Float64)
        msol_l = read(fstream, Int32)
        ca_l = read(fstream, (ComplexF64,(mpert,msol_l,2)))
        msol_r = read(fstream, Int32)
        ca_r = read(fstream, (ComplexF64,(mpert,msol_r,2)))
        e, f, h, m, g, k, eta, rho, taua, taur = read(fstream, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64)
    elseif data_type == 5
        ep = read(fstream,(ComplexF64,mpert))
        et = read(fstream,(ComplexF64,mpert))
        wt = read(fstream,(ComplexF64,(mpert,mpert)))
        efp = read(fstream,(Float64,mpert))
        eft = read(fstream,(Float64,mpert))
        wft = read(fstream,(ComplexF64,(mpert,mpert)))
        wtraw = read(fstream,(ComplexF64,(mpert,mpert)))
    else
        # for all the reads we skipped in the second pass
        print()
    end
end

close(fstream)

idcon_transform(fixtype, sing_flag)
